In [30]:
%load_ext autoreload
%autoreload
import inlp_dataset_handler
import inlp
import inlp_linear_model
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, Ridge, SGDClassifier
from sklearn.svm import LinearSVC, SVR

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
x_train, x_dev = np.random.rand(1000,100) - 0.5, np.random.rand(1000,100) - 0.5
y_train, y_dev = np.sum(x_train, axis = 1) > 0, np.sum(x_dev, axis = 1) > 0

inlp_dataset = inlp_dataset_handler.ClassificationDatasetHandler(x_train, y_train, x_dev, y_dev, dropout_rate = 0, Y_train_main = None, Y_dev_main = None, by_class = False, equal_chance_for_main_task_labels = False)

inlp_model_handler = inlp_linear_model.SKlearnClassifier(LinearSVC, {"dual": False})

In [32]:
P, rowspace_projections, Ws = inlp.run_INLP(num_classifiers = 5, input_dim = 100, is_autoregressive = True, min_accuracy = 0, dataset_handler = inlp_dataset, model = inlp_model_handler)

iteration: 4, accuracy: 0.492: 100%|██████████| 5/5 [00:00<00:00, 21.14it/s]


## Sanity checks

In [33]:
assert np.allclose(P.dot(P), P)
assert np.allclose(Ws[0].dot(P.dot(x_train[0])), 0.0)

for w in Ws:
    for w2 in Ws:
        if w is w2: continue
        assert np.allclose(w.dot(w2.T).item(), 0.0)